In [106]:
# Cryptanalysis - Task 1 - CipherFour and Attack - v2

# TO-DO:

# Remarks:
#  - "^" has to be replaced with "^^" for xor in Sage


import numpy as np

def sbox(input):
    list_out = [6, 4, 12, 5, 0, 7, 2, 14, 1, 15, 3, 13, 8, 10, 9, 11]
    return list_out[input]

def inverse_sbox(input):
    list_out = [6, 4, 12, 5, 0, 7, 2, 14, 1, 15, 3, 13, 8, 10, 9, 11]
    return list_out.index(input)

def sbox_layer(input):
    out = []
    for i in range(4):
        #print((input >> (i * 4) % 16))
        out.append(sbox(((input >> (i * 4)) % 16)))
    res = 0
    for i in range(4):
        res += out[i] * (16 ** i)
    return res

def sbox_layer_inverse(input):
    out = []
    for i in range(4):
        #print(((input >> (i * 4)) % 16))
        out.append(inverse_sbox(((input >> (i * 4)) % 16)))
    res = 0
    for i in range(4):
        res += out[i] * (16 ** i)
    return res

def permutation(inputs):
    inp = inputs
    perm = [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15]
    arr = [0] * 16
    for i in range(16):
        if inp % 2 == 1:
            arr[perm[15 - i]] = 1
        inp = inp >> 1
    return int("".join(str(x) for x in arr), 2)

def inverse_permutation(input):
    inp = input
    perm = [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15]
    arr = [0] * 16
    for i in range(16):
        if inp % 2 == 1:
            arr[perm.index(15 - i)] = 1
        inp = inp >> 1
    return int("".join(str(x) for x in arr), 2)


def encrypt(keys, message):
    ct = message
    for i in range(len(keys) - 2):
        ct ^^= keys[i]
        ct = sbox_layer(ct)
        ct = permutation(ct)
    ct ^^= keys[-2]
    ct = sbox_layer(ct)
    ct ^^= keys[-1]
    return ct

def decrypt(keys, ct):
    pt = ct
    pt ^^= keys[-1]
    pt = sbox_layer_inverse(pt)
    pt ^^= keys[-2]
    for i in range(len(keys) - 3, -1, -1):
        pt = inverse_permutation(pt)
        pt = sbox_layer_inverse(pt)
        pt ^^= keys[i]
    return pt


def generate_pairs_old(diff, number):
    df = "".join("0"*(4-len(bin(x)[2:]))+str(bin(x)[2:]) for x in diff)
    #a=  int("".join(str(bin(x)) for x in diff), 2)
    pairs = []
    singletons = []
    curr = 0
    for m in range(2**16):
        if m not in singletons:
            pairs.append((m, m ^^ diff))
            singletons.append(m)
            singletons.append(m ^^ diff)
            curr += 1
        if curr is not None and curr == number:
            break
    return pairs

def generate_pairs(d, number):
    pairs = []
    singletons = []
    curr = 0
    """
    for i in range(16):
        for j in range(16):
            for k in range(16):
                for l in range(16):
                    m = [i, j, k, l]
                    m_full = l + k * (16) + j * (16)**2 + i * (16)**3
                    if m_full not in singletons:
                        n = [m[0] ^^ diff[0], m[1] ^^ diff[1], m[2] ^^ diff[2], m[3] ^^ diff[3]]
                        n_full = n[3] + n[2] * (16) + n[1] * (16)**2 + n[0] * (16)**3
                        pairs.append((m_full, n_full))
                        singletons.append(m_full)
                        singletons.append(n_full)
                        curr += 1
                    if number is not None and curr == number:
                        break
    """
    
    diff = d[3] + d[2] * (16) + d[1] * (16)**2 + d[0] * (16)**3
    #diff = d[0] + d[1] * (16) + d[2] * (16)**2 + d[3] * (16)**3
    for m in range(16**4):
        if m not in singletons:
            pairs.append((m, m ^^ diff))
            singletons.append(m)
            singletons.append(m ^^ diff)
            curr += 1
        if number is not None and curr == number:
            break
    
    #for m in range(2**16):
    #    if m not in singletons:
    #        pairs.append((m, m ^^ diff))
    #        singletons.append(m)
    #        singletons.append(m ^^ diff)
    #        curr += 1
    #    if curr is not None and curr == number:
    #        break
    return pairs

def attack(keys, filtering):
    diff = [0, 0, 2, 0]
    #diff = [0, 2, 0, 0]
    pairs = generate_pairs(diff, 1000)
    ct_pairs = []
    inv_pairs = []
    for i in range(len(pairs)):
        pair = encrypt(keys, pairs[i][0]), encrypt(keys, pairs[i][1])
        if filtering:
            if (pair[0] ^^ pair[1]) in [1 * 16, 2 * 16, 9 * 16, 10 * 16]:
                ct_pairs.append(pair)
        else:
            ct_pairs.append(pair)
        #inv_pairs.append((inverse_sbox(pair[0]), inverse_sbox(pair[1])))
    #print(ct_pairs)
    #ratio = [0] * 16
    ratio = {}
    
    if filtering:
        print("Survived filtering:", len(ct_pairs), "/", len(pairs))
    
    #guesses = range(16)
    #guesses = [31907, 31908, 31909, 31910, 31911, 31912, 31913]
    guesses = [7*16 - 1 * 16, 7*16, 7*16 + 1 * 16, 7*16 + 2 * 16, 7*16 + 3 * 16]
    guesses = range(0, 16*16, 16)
    #guesses = [6]
    for last_key in guesses:  # should be 16**4
        for i in range(len(ct_pairs)):
            xored_pair = last_key ^^ ct_pairs[i][0], last_key ^^ ct_pairs[i][1]
            inv_pair = (sbox_layer_inverse(xored_pair[0]), sbox_layer_inverse(xored_pair[1]))
            diff = inv_pair[0] ^^ inv_pair[1]
            #print(diff)
            #expected_diff = 12
            #expected_diff = 12 * 16**2
            #expected_diff = 12 * 16
            #expected_diff = 192
            
            expected_diff = 2 * 16
            #expected_diff = 2 * (16)**2
            
            #expected_diff = 4 * 16
            #expected_diff = 4 * 16**2
            
            """
            if filtering:
                if (ct_pairs[i][0] ^^ ct_pairs[i][1]) in [1 * 16, 2 * 16, 9 * 16, 10 * 16]:
                    print("kept")
                    if diff == expected_diff:  # 12
                        if last_key not in ratio:
                            ratio[last_key] = 1
                        else:
                            ratio[last_key] += 1
                else:
                    placeholder = 0
                    print("discarded")
            else:
                if diff == expected_diff:  # 12
                    if last_key not in ratio:
                        ratio[last_key] = 1
                    else:
                        ratio[last_key] += 1
            """
            if diff == expected_diff:  # 12
                if last_key not in ratio:
                    ratio[last_key] = 1
                else:
                    ratio[last_key] += 1
    return ratio

def ints_to_blocks(a_s):
    res = []
    for i in range(len(a_s)):
        out = [0, 0, 0, 0]
        for j in range(4):
            out[3 - j] = ((a_s[i] >> (j * 4)) % 16)
        res.append(out)
    return out

def blocks_to_ints(blocks):
    res = []
    for i in range(len(blocks)):
        a = 0
        for j in range(4):
            a += blocks[i][3 - j] * 16**j
        res.append(a)
    return res
    

#print(7 >> 2)
#keys_arr = [[5, 11, 9, 2], [0, 6, 4, 11], [1, 14, 0, 3], [10, 5, 5, 15], [14, 12, 11, 13], [7, 12, 10, 5]]
#keys_arr = [[5, 11, 9, 2], [0, 6, 4, 11], [1, 14, 0, 3], [10, 5, 5, 15], [14, 12, 11, 13], [7, 12, 10, 5]]
#print(blocks_to_ints(keys_arr))
keys_int = [23442, 1611, 7683, 42335, 60605, 31909]
#print(int("".join(str(x) for x in [1, 0, 1, 1]),2))

#print(decrypt([1,2,3,4,5,6], encrypt([1,2,3,4,5,6],13)))

#print(["0"*(4-len(bin(x)[2:]))+str(bin(x)[2:]) for x in [2]])

#print(attack([1,2,3,4,5,6], False))
#print(attack([1,2,3,4,5,1], False))
#print(attack([23442, 1611, 7683, 42335, 60605, 31909], False))
print(attack([1,2,3,4,5,7*16], False))
print(attack([1,2,3,4,5,7*16], True))

a, b = 37928, 37944
a, b = a ^^ 6, b ^^ 6
a, b = sbox_layer_inverse(a), sbox_layer_inverse(b)
print("a ^^ b:", a ^^ b, "=", int_to_block(a ^^ b))
a, b = a ^^ 5, b ^^ 5
for i in [4, 3, 2, 1]:
    a, b = inverse_permutation(a), inverse_permutation(b)
    a, b = sbox_layer_inverse(a), sbox_layer_inverse(b)
    a, b = a ^^ i, b ^^ i
print(a, b)

#print(generate_pairs([0, 0, 2, 0], 10))

{0: 23, 16: 21, 32: 17, 48: 10, 64: 50, 80: 58, 96: 52, 112: 73, 128: 27, 144: 20, 160: 42, 176: 40, 192: 32, 208: 33, 224: 38, 240: 26}
Survived filtering: 111 / 1000
{0: 23, 16: 21, 32: 17, 48: 10, 64: 50, 80: 58, 96: 52, 112: 73, 128: 27, 144: 20, 160: 42, 176: 40, 192: 32, 208: 33, 224: 38, 240: 26}
a ^^ b: 192 = [0, 0, 12, 0]
6 38
